Création des datasets pour les autres villes sélectionnées dans city_selection. Même pipeline que Paris, adapté ville par ville.


In [ ]:
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


def process_city_from_scratch(filepath, city_name, center_lat, center_lon):
    """
    Nettoie et génère les features pour une ville donnée.
    Espace de features identique à Paris (30 colonnes).
    """
    print(f"\nProcessing : {city_name}")
    df = pd.read_csv(filepath, compression='gzip', low_memory=False)
    
    # 1. Prix
    df['price_clean'] = pd.to_numeric(
        df['price'].astype(str).str.replace(r'[$,]', '', regex=True),
        errors='coerce'
    )
    df = df.dropna(subset=['price_clean'])

    # 2. Distance au centre
    df['dist_raw'] = np.sqrt((df['latitude'] - center_lat)**2 + (df['longitude'] - center_lon)**2)
    city_radius = df['dist_raw'].quantile(0.95)
    df['dist_to_center'] = np.clip(df['dist_raw'] / city_radius, 0, 1)

    # 3. Salles de bain
    def clean_bath(text):
        if pd.isna(text): return 1.0
        text = str(text).lower()
        if 'half' in text or 'demi' in text: return 0.5
        res = re.findall(r"(\d+(\.\d+)?)", text)
        return float(res[0][0]) if res else 1.0

    if 'bathrooms_text' in df.columns:
        df['bathrooms_qty'] = df['bathrooms_text'].apply(clean_bath)
    else:
        df['bathrooms_qty'] = df.get('bathrooms', 1.0).fillna(1.0)

    # 4. Amenities
    if 'amenities' in df.columns:
        df['amen_str'] = df['amenities'].astype(str).str.lower()
        
        power_map = {
            'has_ac': 'air conditioning|clim',
            'has_elevator': 'elevator|lift|ascenseur',
            'has_dishwasher': 'dishwasher|lave-vaisselle',
            'has_parking': 'parking',
            'has_balcony': 'balcony|terrace|patio|balcon|terrasse',
            'has_washing_machine': 'washer|lave-linge'
        }
        for col, regex in power_map.items():
            df[col] = df['amen_str'].str.contains(regex, regex=True).astype(int)
            
        def get_thermal(text):
            if 'air conditioning' in text or 'clim' in text: return 2
            if 'fan' in text or 'ventilateur' in text: return 1
            return 0
        df['standing_thermal'] = df['amen_str'].apply(get_thermal)

        themes = {
            'score_chef_kitchen': ['stove', 'oven', 'refrigerator', 'freezer', 'wine glasses'],
            'score_coffee': ['coffee maker', 'espresso', 'nespresso'],
            'score_wellness': ['pool', 'sauna', 'jacuzzi', 'bathtub'],
            'score_work': ['wifi', 'dedicated workspace', 'monitor'],
            'score_view': ['view', 'skyline', 'waterfront'],
            'score_secure': ['lockbox', 'self check-in']
        }
        for score_name, keywords in themes.items():
            df[score_name] = df['amen_str'].apply(lambda x: sum(1 for k in keywords if k in x))

    # 5. Quartier
    if 'neighbourhood_cleansed' in df.columns:
        freq = df['neighbourhood_cleansed'].value_counts(normalize=True)
        df['neigh_popularity'] = df['neighbourhood_cleansed'].map(freq).fillna(0)

    # 6. Encodage type de logement
    if 'property_type' in df.columns:
        def group_prop(x):
            x = str(x).lower()
            if 'entire' in x or 'apt' in x: return 'entire_unit'
            if 'private' in x: return 'private_room'
            if 'hotel' in x: return 'hotel'
            return 'other'
        df['prop_group'] = df['property_type'].apply(group_prop)
        df = pd.get_dummies(df, columns=['prop_group'], prefix='pg', drop_first=False)

    if 'room_type' in df.columns:
        df = pd.get_dummies(df, columns=['room_type'], prefix='rt', drop_first=False)

    for col in df.columns:
        if col.startswith('pg_') or col.startswith('rt_'):
            df[col] = df[col].astype(bool)

    # 7. Label et cible
    df['city_label'] = city_name
    df['target_class'] = pd.qcut(df['price_clean'], q=4, labels=[0, 1, 2, 3], duplicates='drop').astype(int)

    # 8. Alignement des features sur Paris (whitelist stricte)
    PARIS_FEATURES = [
        'accommodates', 'bedrooms', 'beds', 'minimum_nights_avg_ntm', 'number_of_reviews', 
        'price_clean', 'dist_to_center', 'bathrooms_qty', 'has_ac', 'has_elevator', 
        'has_dishwasher', 'has_parking', 'has_balcony', 'has_washing_machine', 
        'standing_thermal', 'score_chef_kitchen', 'score_coffee', 'score_wellness', 
        'score_work', 'score_view', 'score_secure', 'neigh_popularity', 
        'pg_entire_unit', 'pg_hotel', 'pg_private_room', 
        'rt_Entire home/apt', 'rt_Private room', 'rt_Shared room', 
        'city_label', 'target_class'
    ]

    # Colonnes manquantes -> on les crée à 0
    for col in PARIS_FEATURES:
        if col not in df.columns:
            if col.startswith('pg_') or col.startswith('rt_') or col.startswith('has_'):
                df[col] = False
            else:
                df[col] = 0.0

    df_final = df[PARIS_FEATURES].copy()
    
    num_cols = df_final.select_dtypes(include=[np.number]).columns
    df_final[num_cols] = df_final[num_cols].fillna(df_final[num_cols].median())

    print(f"   Shape finale : {df_final.shape}")
    return df_final


def audit_city_data(df, city_name):
    """Vérifie la qualité d'un dataset."""
    print(f"\n--- Audit : {city_name} ---\n")
    
    # 1. Valeurs manquantes
    print("1. Valeurs manquantes et types")
    null_counts = df.isnull().sum()
    if null_counts.sum() == 0:
        print("   Aucun NaN")
    else:
        print("   Il reste des NaNs :")
        print(null_counts[null_counts > 0])
        
    print("\n   Types :")
    print(df.dtypes.value_counts())

    # 2. Equilibre des classes
    print("\n2. Equilibre des classes")
    target_dist = df['target_class'].value_counts(normalize=True).sort_index()
    print(target_dist)
    if target_dist.min() > 0.20:
        print("   Classes équilibrées")
    else:
        print("   Déséquilibre détecté")

    # 3. Corrélations
    print("\n3. Corrélations avec le prix")
    numeric_df = df.select_dtypes(include=[np.number])
    corr_matrix = numeric_df.corr()['price_clean'].drop('price_clean').sort_values(ascending=False)
    
    print("Top 5 positives :")
    print(corr_matrix.head(5))
    print("\nTop 5 négatives :")
    print(corr_matrix.tail(5))


In [ ]:
# Rome - Panthéon / Piazza Navona
df_rome = process_city_from_scratch(
    filepath='../data/rome_listings_raw_2025-09-14.csv.gz', 
    city_name='Rome', 
    center_lat=41.8986, 
    center_lon=12.4768
)
df_rome.to_csv('../data/rome_dataset_final_ready.csv.gz', index=False, compression='gzip')

# Madrid - Puerta del Sol
df_madrid = process_city_from_scratch(
    filepath='../data/madrid_listings_raw_2025-09-14.csv.gz', 
    city_name='Madrid', 
    center_lat=40.4168, 
    center_lon=-3.7038
)
df_madrid.to_csv('../data/madrid_dataset_final_ready.csv.gz', index=False, compression='gzip')

# Vienne - Stephansplatz
df_vienna = process_city_from_scratch(
    filepath='../data/vienna_listings_raw_2025-09-14.csv.gz', 
    city_name='Vienna', 
    center_lat=48.2082, 
    center_lon=16.3738
)
df_vienna.to_csv('../data/vienna_dataset_final_ready.csv.gz', index=False, compression='gzip')



--- 🏭 DÉMARRAGE DU PROCESSING POUR : ROME ---
✅ Processing terminé pour Rome ! Shape finale : (33564, 30)

--- 🏭 DÉMARRAGE DU PROCESSING POUR : MADRID ---
✅ Processing terminé pour Madrid ! Shape finale : (18953, 30)

--- 🏭 DÉMARRAGE DU PROCESSING POUR : VIENNA ---
✅ Processing terminé pour Vienna ! Shape finale : (10306, 30)


In [19]:
# Audits
audit_city_data(df_rome, "Rome")


   🔍 AUDIT DU DATASET : ROME 

=== 1. AUDIT TECHNIQUE (Valeurs manquantes & Types) ===
✅ Aucun NaN détecté. Dataset 100% rempli.

Types de colonnes :
int64      16
float64     7
bool        6
str         1
Name: count, dtype: int64

=== 2. AUDIT DE L'ÉQUILIBRE (Target) ===
target_class
0    0.250566
1    0.254171
2    0.245263
3    0.250000
Name: proportion, dtype: float64
✅ Classes bien équilibrées (Effet Quartiles).

=== 3. AUDIT DU SIGNAL (Corrélations avec le Prix) ===
Top 5 Corrélations Positives (Pousse le prix à la hausse) :
target_class     0.294808
bathrooms_qty    0.219375
bedrooms         0.193825
accommodates     0.193774
beds             0.161821
Name: price_clean, dtype: float64

Top 5 Corrélations Négatives (Pousse le prix à la baisse) :
score_work          -0.004857
score_secure        -0.013456
has_parking         -0.033715
number_of_reviews   -0.035141
dist_to_center      -0.081937
Name: price_clean, dtype: float64

✅ Audit terminé.


In [20]:
audit_city_data(df_madrid, "Madrid")


   🔍 AUDIT DU DATASET : MADRID 

=== 1. AUDIT TECHNIQUE (Valeurs manquantes & Types) ===
✅ Aucun NaN détecté. Dataset 100% rempli.

Types de colonnes :
int64      16
float64     7
bool        6
str         1
Name: count, dtype: int64

=== 2. AUDIT DE L'ÉQUILIBRE (Target) ===
target_class
0    0.252572
1    0.250462
2    0.250567
3    0.246399
Name: proportion, dtype: float64
✅ Classes bien équilibrées (Effet Quartiles).

=== 3. AUDIT DU SIGNAL (Corrélations avec le Prix) ===
Top 5 Corrélations Positives (Pousse le prix à la hausse) :
target_class     0.222395
accommodates     0.146132
bedrooms         0.110695
beds             0.104582
bathrooms_qty    0.080566
Name: price_clean, dtype: float64

Top 5 Corrélations Négatives (Pousse le prix à la baisse) :
score_work               -0.004831
number_of_reviews        -0.012929
has_parking              -0.023988
minimum_nights_avg_ntm   -0.026565
dist_to_center           -0.054975
Name: price_clean, dtype: float64

✅ Audit terminé.


In [21]:
audit_city_data(df_vienna, "Vienna")


   🔍 AUDIT DU DATASET : VIENNA 

=== 1. AUDIT TECHNIQUE (Valeurs manquantes & Types) ===
✅ Aucun NaN détecté. Dataset 100% rempli.

Types de colonnes :
int64      16
float64     7
bool        6
str         1
Name: count, dtype: int64

=== 2. AUDIT DE L'ÉQUILIBRE (Target) ===
target_class
0    0.258296
1    0.248593
2    0.243839
3    0.249272
Name: proportion, dtype: float64
✅ Classes bien équilibrées (Effet Quartiles).

=== 3. AUDIT DU SIGNAL (Corrélations avec le Prix) ===
Top 5 Corrélations Positives (Pousse le prix à la hausse) :
target_class     0.217671
bedrooms         0.095400
accommodates     0.080376
beds             0.078214
bathrooms_qty    0.047502
Name: price_clean, dtype: float64

Top 5 Corrélations Négatives (Pousse le prix à la baisse) :
neigh_popularity    -0.025623
has_parking         -0.034790
score_work          -0.040497
number_of_reviews   -0.044962
score_secure        -0.075508
Name: price_clean, dtype: float64

✅ Audit terminé.


In [22]:
path_paris = "../data/paris_dataset_final_ready.csv.gz"
df_paris = pd.read_csv(path_paris, compression='gzip')
audit_city_data(df_paris, "Paris")


   🔍 AUDIT DU DATASET : PARIS 

=== 1. AUDIT TECHNIQUE (Valeurs manquantes & Types) ===
✅ Aucun NaN détecté. Dataset 100% rempli.

Types de colonnes :
int64      16
float64     7
bool        6
str         1
Name: count, dtype: int64

=== 2. AUDIT DE L'ÉQUILIBRE (Target) ===
target_class
0    0.261438
1    0.238664
2    0.250291
3    0.249607
Name: proportion, dtype: float64
✅ Classes bien équilibrées (Effet Quartiles).

=== 3. AUDIT DU SIGNAL (Corrélations avec le Prix) ===
Top 5 Corrélations Positives (Pousse le prix à la hausse) :
target_class     0.681494
bedrooms         0.465915
accommodates     0.465111
bathrooms_qty    0.409981
beds             0.310000
Name: price_clean, dtype: float64

Top 5 Corrélations Négatives (Pousse le prix à la baisse) :
score_view                0.032500
minimum_nights_avg_ntm   -0.064034
number_of_reviews        -0.080754
neigh_popularity         -0.088824
score_secure             -0.091487
Name: price_clean, dtype: float64

✅ Audit terminé.
